# 1) Titolo e obiettivi
Lezione 35: Information Retrieval con TF-IDF e similarita' coseno.

---

## Mappa concettuale della lezione

```
INFORMATION RETRIEVAL - PIPELINE COMPLETA
==========================================

    +--------------------+
    |   CORPUS (N docs)  |
    |  "doc1", "doc2"... |
    +----------+---------+
               |
               v
    +--------------------+      +-----------------+
    | TfidfVectorizer    |      | Vocabolario V   |
    | .fit(corpus)       +----->| fit salva i     |
    | .transform(corpus) |      | termini => REUSE|
    +----------+---------+      +-----------------+
               |
               v
    +--------------------+
    |  MATRICE TF-IDF    |
    |  (N docs x V feat) |
    +----------+---------+
               |
               |    +------------------+
               |    |  QUERY "q"       |
               |    |  .transform([q]) |
               |    +--------+---------+
               |             |
               v             v
    +----------------------------+
    | cosine_similarity(Q, M)    |
    | => scores (N,)             |
    +-------------+--------------+
                  |
                  v
    +----------------------------+
    |  RANKING: argsort desc     |
    |  top_k = indices[:k]       |
    +-------------+--------------+
                  |
                  v
    +----------------------------+
    |  VALUTAZIONE: precision@k  |
    |  = |rilevanti ∩ top_k| / k |
    +----------------------------+
```

---

## Obiettivi didattici

| # | Obiettivo | Livello |
|---|-----------|---------|
| 1 | Comprendere la pipeline IR: indicizzazione, query, ranking | Fondamentale |
| 2 | Usare TfidfVectorizer per indicizzare documenti e query | Operativo |
| 3 | Applicare cosine_similarity per rankare risultati | Operativo |
| 4 | Valutare la qualita' con precision@k | Valutativo |
| 5 | Implementare query expansion per migliorare recall | Avanzato |
| 6 | Gestire stopword di dominio e preprocessing coerente | Best Practice |

---

## Concetti chiave

> **Information Retrieval (IR)**: disciplina che si occupa di trovare documenti rilevanti rispetto a un bisogno informativo espresso come query testuale.

> **Modello vettoriale**: rappresenta sia documenti che query come vettori nello stesso spazio; la similarita' coseno misura la "vicinanza angolare" tra essi.

> **Precision@k**: metrica che risponde a "Quanti dei primi k risultati sono effettivamente rilevanti?" - cruciale quando l'utente guarda solo i top risultati.

---

## Tassonomia dei sistemi IR

```
+---------------------------+---------------------------+
|   IR TRADIZIONALE         |   IR SEMANTICO            |
+---------------------------+---------------------------+
| TF-IDF / BM25             | Embeddings (Word2Vec/BERT)|
| Match lessicale esatto    | Match concettuale         |
| Vocabolario fisso         | Spazio denso latente      |
| Interpretabile            | Black-box ma potente      |
| Baseline veloce           | Richiede modelli pesanti  |
+---------------------------+---------------------------+
```

Questa lezione si concentra sull'IR tradizionale con TF-IDF, che rimane una **baseline solida e interpretabile** prima di passare a metodi semantici.

---

## Cosa useremo
- `TfidfVectorizer` per indicizzare documenti e trasformare query
- `cosine_similarity` per calcolare rilevanza
- `pandas` per organizzare risultati
- Funzioni custom per precision@k e query expansion

## Prerequisiti
- Concetti di BoW e TF-IDF (Lezione 31)
- Similarita' coseno (Lezione 31)
- Preprocessing testuale (Lezione 30)


# 2) Teoria concettuale
- Information Retrieval: trovare documenti rilevanti dato un bisogno informativo (query).
- TF-IDF: rappresenta documenti e query come vettori pesati per frequenza/rarita'.
- Similarita' coseno: misura di vicinanza tra vettori; usata per rankare i documenti.
- Metriche: precision@k (quota di documenti rilevanti nei primi k).


# 3) Schema mentale / mappa decisionale
1. Raccogli e pulisci documenti (stopword, lower).
2. Fit TF-IDF sul corpus; salva vocabolario.
3. Trasforma query con lo stesso vectorizer.
4. Calcola similarita' coseno e ordina i documenti.
5. Valuta con precision@k se hai rilevanza; altrimenti spot-check manuale.
6. Migliora con query expansion o stopword custom.


# 4) Sezione dimostrativa
Demo: corpus sintetico, indicizzazione TF-IDF, ricerca con due query, calcolo precision@k e query expansion minimale.


In [ ]:
# Setup librerie
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)


In [ ]:
# Corpus di esempio e label di rilevanza per valutazione
corpus = [
    "Guida all'uso delle API aziendali",
    "Manuale di marketing digitale per e-commerce",
    "Report finanziario trimestrale",
    "Procedura per richiedere rimborsi e resi",
    "Tutorial machine learning applicato al churn"
]
# Rilevanza binaria per due query
relevance = {
    'api': {0:1},
    'rimborsi': {3:1}
}
print(f"Documenti: {len(corpus)}")
assert len(corpus)>0


In [ ]:
# Indicizzazione TF-IDF
vectorizer = TfidfVectorizer(stop_words='italian')
X = vectorizer.fit_transform(corpus)
print(f"Shape TF-IDF: {X.shape}, sparsimita': {1 - X.nnz/(X.shape[0]*X.shape[1]):.1%}")


In [ ]:
# Funzione di ricerca con precision@k

def search(query, top_k=3):
    q_vec = vectorizer.transform([query])
    sims = cosine_similarity(q_vec, X).flatten()
    order = sims.argsort()[::-1][:top_k]
    return order, sims[order]

def precision_at_k(query, k):
    order, _ = search(query, top_k=k)
    rel_docs = relevance.get(query, {})
    rel_found = sum(1 for idx in order if rel_docs.get(idx,0)==1)
    return rel_found / k


In [ ]:
# Eseguiamo due query
for q in ['api','rimborsi']:
    order, scores = search(q, top_k=3)
    print(f"Query: {q}")
    for idx, score in zip(order, scores):
        print(f"  Score {score:.3f} -> Doc {idx}: {corpus[idx]}")
    print(f"Precision@3: {precision_at_k(q,3):.2f}")


In [ ]:
# Query expansion semplice: sinonimi manuali
synonyms = {'api': ['api','interfaccia'], 'rimborsi': ['rimborsi','resi','refund']}

def expand_query(q):
    terms = synonyms.get(q, [q])
    return ' '.join(terms)

q_expanded = expand_query('rimborsi')
order, scores = search(q_expanded, top_k=3)
print("Query espansa:", q_expanded)
print("Precision@3 espansa:", precision_at_k('rimborsi',3))


### Osservazioni
- Precision@k migliora se i termini espansi coprono il vocabolario.
- Stopword influenzano la sparsita' e i match; controllarle per il dominio.


# 5) Esercizi svolti (passo-passo)
## Esercizio 35.1 - Aggiungere documenti e rivalutare


In [ ]:
# Esercizio 35.1
new_doc = "Documentazione API per pagamenti"
corpus2 = corpus + [new_doc]
vectorizer.fit(corpus2)
order, scores = search('api', top_k=3)
print("Nuovi top3 per api:")
for idx, score in zip(order, scores):
    print(idx, score, corpus[idx] if idx < len(corpus) else '')


## Esercizio 35.2 - Stopword di dominio


In [ ]:
# Esercizio 35.2
custom_stop = ['manuale','guida']
vec_custom = TfidfVectorizer(stop_words=set(custom_stop + list(TfidfVectorizer(stop_words='italian').get_stop_words())))
Xc = vec_custom.fit_transform(corpus)
print(f"Sparsimita' con stopword custom: {1 - Xc.nnz/(Xc.shape[0]*Xc.shape[1]):.1%}")


## Esercizio 35.3 - Precision@k con rilevanze multiple


In [ ]:
# Esercizio 35.3
relevance_multi = {'marketing': {1:1, 4:1}}
q = 'marketing'
order, _ = search(q, top_k=3)
rel_docs = relevance_multi[q]
prec3 = sum(1 for idx in order if rel_docs.get(idx,0)==1)/3
print(f"Precision@3 per {q}: {prec3:.2f}")


# 6) Conclusione operativa - Bignami IR

---

## I 5 Take-Home Messages

| # | Concetto | Perche' conta |
|---|----------|---------------|
| 1 | **Pipeline IR = fit(corpus) + transform(query) + cosine + rank** | Workflow standard per qualsiasi ricerca testuale |
| 2 | **Stesso vectorizer per corpus e query** | Vocabolario deve essere identico, mai rifittare |
| 3 | **Precision@k misura qualita' percepita** | Utenti guardano solo i primi risultati |
| 4 | **Query expansion aumenta recall** | Aggiungere sinonimi trova documenti con termini diversi |
| 5 | **Stopword di dominio migliorano precision** | Rimuovere termini troppo comuni nel contesto specifico |

---

## Pipeline IR in 6 step

```
STEP 1: CORPUS              STEP 2: PREPROCESSING
+----------------+          +--------------------+
| docs = [...]   |   --->   | lower, stopword,   |
|                |          | stemming/lemma     |
+----------------+          +--------------------+
                                    |
                                    v
STEP 3: INDICIZZAZIONE      STEP 4: QUERY
+--------------------+      +--------------------+
| vectorizer.fit()   |      | q_vec = transform  |
| doc_matrix = tfm() |      | ([query])          |
+--------------------+      +--------------------+
                                    |
                                    v
STEP 5: RANKING             STEP 6: VALUTAZIONE
+--------------------+      +--------------------+
| scores = cosine()  |      | precision@k        |
| top_k = argsort()  |      | recall@k, MRR      |
+--------------------+      +--------------------+
```

---

## Confronto metriche IR

| Metrica | Formula | Interpretazione |
|---------|---------|-----------------|
| Precision@k | rilevanti_in_k / k | Quanti top-k sono buoni? |
| Recall@k | rilevanti_in_k / tot_rilevanti | Quanti rilevanti ho trovato? |
| MRR | 1/rank_primo_rilevante | Quanto presto trovo qualcosa? |
| MAP | media precision a ogni rilevante | Qualita' globale del ranking |

---

## Template operativo

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1) Setup
corpus = ["doc1 text", "doc2 text", ...]
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9)
doc_matrix = vectorizer.fit_transform(corpus)

# 2) Funzione ricerca
def search(query, top_k=5):
    q_vec = vectorizer.transform([query])
    scores = cosine_similarity(q_vec, doc_matrix).flatten()
    top_idx = scores.argsort()[::-1][:top_k]
    return [(i, corpus[i], scores[i]) for i in top_idx]

# 3) Query expansion
def expand_query(query, synonyms_dict):
    tokens = query.lower().split()
    expanded = tokens.copy()
    for t in tokens:
        if t in synonyms_dict:
            expanded.extend(synonyms_dict[t])
    return ' '.join(expanded)

# 4) Precision@k
def precision_at_k(retrieved_idx, relevant_set, k):
    top_k = retrieved_idx[:k]
    hits = len(set(top_k) & relevant_set)
    return hits / k
```

---

## Errori comuni e soluzioni

| Errore | Conseguenza | Soluzione |
|--------|-------------|-----------|
| Rifittare vectorizer su ogni query | Vocabolario diverso, risultati sbagliati | Salvare vectorizer dopo fit su corpus |
| Query non preprocessata | Termini non matchano | Stesso preprocessing di corpus |
| Ignorare stopword di dominio | Termini troppo comuni dominano | Stopword custom + max_df |
| Nessuna valutazione | Non sai se funziona | Creare set di rilevanze anche piccolo |

---

## Metodi e attributi chiave

| Metodo/Attributo | Uso |
|------------------|-----|
| `vectorizer.fit_transform(corpus)` | Indicizza corpus |
| `vectorizer.transform([query])` | Trasforma query |
| `vectorizer.vocabulary_` | Mappa termine -> indice |
| `cosine_similarity(q, M)` | Scores di similarita' |
| `np.argsort()[::-1][:k]` | Top-k indici |


# 7) Checklist di fine lezione
- [ ] Ho pulito il corpus e definito stopword adeguate.
- [ ] Ho fit il TF-IDF sul corpus e usato transform sulle query.
- [ ] Ho calcolato similarita' coseno e ordinato i documenti.
- [ ] Ho valutato precision@k dove avevo rilevanze.
- [ ] Ho considerato query expansion o sinonimi di dominio.

Glossario
- Information Retrieval: recupero di documenti rilevanti da un corpus.
- Precision@k: quota di documenti rilevanti nei primi k.
- Query expansion: aggiunta di termini correlati alla query.
- Similarita' coseno: misura di vicinanza tra vettori.
- Vocabolario: termini indicizzati dal vectorizer.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-XX | Struttura iniziale 8 sezioni |
| 2.0 | 2024-12-XX | Espansione completa con pipeline IR ASCII |
| 2.1 | - | Aggiunta tassonomia IR tradizionale vs semantico |
| 2.2 | - | Tabella obiettivi con livelli di competenza |
| 2.3 | - | 5 take-home messages con razionali |
| 2.4 | - | Confronto metriche IR (P@k, R@k, MRR, MAP) |
| 2.5 | - | Template completo con search + expand + precision |
| 2.6 | - | Tabella errori comuni con soluzioni |

---

## Note di versione

**v2.0 - Espansione didattica completa**
- Pipeline IR visualizzata in ASCII con tutti i passaggi
- Tassonomia che posiziona TF-IDF rispetto a metodi semantici
- Emphasis su "stesso vectorizer" come punto critico
- Metriche IR spiegate con formule e interpretazioni
- Template operativo con 4 funzioni chiave
- Preparazione concettuale per IR semantico (Lezione 36+)

**Dipendenze didattiche**
- Richiede: Lezione 30 (preprocessing), Lezione 31 (TF-IDF)
- Prepara: Lezione 36 (Knowledge Mining), semantic search
